<h3>Web Scrapping</h3>
<h4>website-Shiksha.com

importing the library


In [20]:
import csv
import requests
import time
from bs4 import BeautifulSoup
import time


In [6]:
start =time.time()
def check_null(data):
    if data:
        return data.text
    else:
        return None
columns = ["Sr_no", "College Name", "Location", "Courses offered", "Exam accepted", "Average Package", "Total Fees Range"]
index = 1

with open('shiksha.csv', mode='a', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(columns)

    page_number = 1  # Initialize page_number
    while page_number <= 809:
        url = f"https://www.shiksha.com/search?pn={page_number}&q=list%20of%20colleges"
        # Providing it for accessing the website
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.111 Safari/537.3"
        }
        response = requests.get(url, headers=headers)
        # checking if connection is there
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            # Extracting 
            college_name = soup.find_all(class_="cd76") #college_name
            courses_offered = soup.find_all(class_="_9865 ripple dark") #courses offered
            location = soup.find_all(class_="_5588") #college location
            exam_accepted = soup.find_all(class_='_0954')   #exam accepted
            container = soup.find_all("div", class_='cd4f _5c64 contentColumn_2') # container it contains both average fees and placement

            for name, loc, course, exam, contain in zip(college_name, location, courses_offered, exam_accepted, container):
                # Extracting total fees and average packages from container
                total_fees = check_null(contain.find_all_next('div', class_="dcfd undefined")[2])
                average_package = check_null(contain.find_next('a', class_="ripple dark"))
                # Writing in csv file
                csv_writer.writerow([index, name.text, loc.text, course.text, exam.text.split(" "),
                                    average_package, total_fees])
                index += 1

            page_number += 1
      
        else:
            print("Failed to access the website:", response.status_code)
            break
end = time.time()
execution_time = end - start
print(f"Execution time: {execution_time} seconds")


In [7]:
import pandas as pd
df=pd.read_csv("shiksha.csv")
df.drop_duplicates(subset=['College Name'],inplace=True)


In [8]:
df.count()

Sr_no               24210
College Name        24210
Location            24210
Courses offered     24210
Exam accepted       24210
Average Package      7518
Total Fees Range    24210
dtype: int64

In [14]:
df.describe()

,Sr_no
count,24210.000000
mean,12112.276043
std,6993.491555
min,1.000000
25%,6056.250000
50%,12112.500000
75%,18168.750000
max,24227.000000


<h3> Uploading data to hdfs using pyspark

In [ ]:
from pyspark.sql import SparkSession
import os

setting ip address

In [ ]:
os.environ["SPARK_IP"]="10.0.2.15"

In [ ]:
spark=SparkSession.builder.appName("webscraping").getOrCreate()

In [ ]:
try:
    df=spark.read.csv("/home/hdoop/Downloads/college_dunia.csv",inferSchema=True,header=True)
    new_df=df.dropDuplicates()
    new_df.write.format("csv").save('hdfs://localhost:9000/web_scraper/shiksha_scrapped_data.csv',header=True,inferschema=True)
except Exception as e:
    print(e)